# Calculate O2-O2_bar, T-T_bar, S-S_bar

In [1]:
import Phi_derivative as Pd
import xarray as xr
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']

## T_mean, S_mean, O2_mean

In [3]:
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]
filename = f'{path}/DPLE_driftcorrected_O2_mean_ens_mean.nc'
ds1 = xr.open_dataset(filename)
O2_mean = ds1['O2_mean'].isel(Y=0, z_t=range(0, 35), nlat=range(a1, a2), nlon=range(b1, b2))
filename = f'{path}/DPLE_driftcorrected_TEMP_mean_ens_mean.nc'
ds1 = xr.open_dataset(filename)
T_mean = ds1['TEMP'].isel(Y=0, z_t=range(0, 35), nlat=range(a1, a2), nlon=range(b1, b2))
filename = f'{path}/DPLE_driftcorrected_SALT_mean_ens_mean.nc'
ds1 = xr.open_dataset(filename)
S_mean = ds1['SALT'].isel(Y=0, z_t=range(0, 35), nlat=range(a1, a2), nlon=range(b1, b2))
filename = f'{path}/DPLE_driftcorrected_pO2_mean_dimension_reduced_ens_mean_monthly.nc'
ds1 = xr.open_dataset(filename)
depth = ds1['3ddepth']

## Metabolic Traits Ac, Eo

In [4]:
import traits_db
df = traits_db.df_mi
df.head()

,Species,Phylum,Ao,Ac,Eo
0,Acanthephyra acutifrons,Crustacea,26.809651,13.364732,-0.0278
1,Acanthephyra curtirostris,Crustacea,25.575448,14.364194,0.0380
2,Acanthephyra purpurea,Crustacea,23.310023,7.460879,0.1320
3,Acanthephyra smithi,Crustacea,28.985507,17.657939,-0.0301
5,Acipenser brevirostrum,Chordata,22.123894,4.901716,0.3626


In [5]:
Ac_dist = np.percentile(df.Ac, [50])
#Ao_dist = np.percentile(df.Ao, [25, 50, 75])
Eo_dist = np.percentile(df.Eo, [25, 50, 75])
print(Ac_dist)
#print(Ao_dist)
print(Eo_dist)

[7.34832416]
[0.1985 0.3389 0.5625]


## calculate the partial derivatives

In [6]:
%%time
nrow = len(Ac_dist)
ncol = len(Eo_dist)
from itertools import product
dss = xr.Dataset()

for i, j in product(range(nrow), range(ncol)):
    Ac = Ac_dist[i]
    Eo = Eo_dist[j]
    print(Ac,Eo)
    dss[f'dPhidO2_{nrow*i+j}'] =  Pd.dPhidO2(O2_mean,T_mean,S_mean,depth,Ac,Eo,0.022) #dEodT=0.022
    print('dPhidO2 done')
    dss[f'dPhidT_{nrow*i+j}'] = Pd.dPhidT(O2_mean,T_mean,S_mean,depth,Ac,Eo,0.022)  #dEodT=0.022
    print('dPhidT done')
    dss[f'dPhidS_{nrow*i+j}'] = Pd.dPhidS(O2_mean,T_mean,S_mean,depth,Ac,Eo,0.022)  #dEodT=0.022
    print('dPhidS done')

7.3483241595629805 0.1985
dPhidO2 done
dPhidT done
dPhidS done
7.3483241595629805 0.3389
dPhidO2 done
dPhidT done
dPhidS done
7.3483241595629805 0.5625
dPhidO2 done
dPhidT done
dPhidS done
CPU times: user 33min 6s, sys: 22min 39s, total: 55min 46s
Wall time: 1h 4min 53s


In [7]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dss.load()

<xarray.Dataset>
Dimensions:    (z_t: 35, nlat: 140, nlon: 195, L: 122)
Coordinates:
  * z_t        (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT       (nlat, nlon) float64 14.96 14.96 14.97 ... 70.43 70.42 70.42
    ULONG      (nlat, nlon) float64 101.7 102.9 104.0 ... 318.9 319.5 320.0
    ULAT       (nlat, nlon) float64 15.13 15.13 15.14 ... 70.64 70.63 70.63
    TLONG      (nlat, nlon) float64 101.2 102.3 103.4 ... 318.6 319.2 319.7
    Y          int64 1954
  * L          (L) int64 1 2 3 4 5 6 7 8 9 ... 115 116 117 118 119 120 121 122
Dimensions without coordinates: nlat, nlon
Data variables:
    dPhidO2_0  (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dPhidT_0   (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dPhidS_0   (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dPhidO2_1  (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dPhidT_1   (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dPhidS_1   (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dPhidO2_2  (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dPhidT_2   (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    dPhidS_2   (L, z_t, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan

In [8]:
%%time
dss.to_netcdf(f'{dout}/DPLE_driftcorrected_dPhidO2_dPhidT_dPhidS_ens_mean_monthly.nc', mode='w')

CPU times: user 78.2 ms, sys: 3.16 s, total: 3.23 s
Wall time: 3.84 s
